In [1]:
# libraries
import pandas as pd
import os
import sys

# modules
sys.path.append(os.path.abspath("modules"))
import modules.dictionary as dict
from modules.classify import classify_keywords

# set your path here
os.chdir("input_data")
print(f'current directory: {os.getcwd()}')

current directory: /Users/tomokitakata/Desktop/research/dev/muldistribution/input_data


In [2]:
# target word(s) : class(es) assigned

override_map_2014 = {

    # ----内科・外科----
    ("訪問診療", "訪問診療科"): ["一般内科"],
    ("漢方", "漢方専門", "漢方科", "漢方（思）", "漢方（不）", "漢方医学科", "漢方診療科", "漢方外来"): ["一般内科"], 
    ("総合診療", "総合診療科", "女性診療外来", "医療科", "診療科"): ['一般内科', '一般外科'],
    ("基本診療", "基本診療科", "全科", "健診等", "健診", "検診科"): ['一般内科', '一般外科'],

     # ----消化器内科・外科----
    ("消科", "消化器", "消火器科", "消化器科", "消化器科（胃）", "（消化器"): ["消化器内科", "消化器外科"],
    ("胃腸科", "胃腸下科", "胃腸器科", "消・胃腸科", "消胃", "胃腸科（消化器科）"): ["消化器内科", "消化器外科"],
    ("消化器・胃腸科", "消化器胃腸科", "消化器科（胃腸科）"): ["消化器内科", '消化器外科'],

     # ----呼吸器内科・外科----
    "呼吸器科": ["呼吸器内科", '呼吸器外科'],
    ("喘息外来"): ['呼吸器内科'],

    # ----気管食道内科・外科----
    ("気管食道科", "気食", "気管食道", "気管科", "気管支科", "気道食道科", "気管", "食道科", "気管食科", "気管食道科", "気道食道科", "気管科", "気管支科",): ["気管食道内科", "気管食道外科"],

     # ----循環器内科・外科----
    ("循環器科", "循環器", "循環科"): ["循環器内科", "循環器外科"],
    "循呼": ["呼吸器内科", '呼吸器外科', "循環器内科", "循環器外科"],
    "禁煙外来": ["循環器内科"],

    # ----精神・心療----
    "精心内": ["精神科", "心療内科"],#
    "子どものこころ診療科": ['心療内科'],
    "心身医療科": ['心療内科'],

    "女性不妊症": ['産婦人科'],
    "疼": ['麻酔科', '整形外科', 'リハビリテーション科'],
    "成分献血": ['病理診断科'],#
    
    # ----肝臓・胆嚢・膵臓----
    ("肝臓科"): ['肝・胆・膵内科', '肝・胆・膵外科'],
    "胆のう": ['肝・胆・膵内科', '肝・胆・膵外科'],
    "肝臓・胆のう・膵臓科": ['肝・胆・膵内科', '肝・胆・膵外科'],

    # ----老年科----
    ("老年期認知症外来", "老年期認知症診療科", "老年期認知症科", "もの忘れこころ外来"): ['老年科'],

    # ----リハビリ----
    ("脳神経リハビリテーション科", "脳神経外科リハビリテーション科"): ['リハビリテーション科'],

    # ----癌----
    "が内": ['血液・腫瘍内科'], # おそらく「がん内科」 Oncology/Internal Medicine for Cancer Treatment

    # ----歯科----
    "矯科": ["歯科・口腔顎顔面外科"], #おそらく「矯正歯科」 Orthodontics
    "噛み合わせ不良による頭痛": ['歯科・口腔顎顔面外科'],

    # ----小児科----
    "児童思春期外来": ['小児科'],

    # ----腎臓・内分泌----
    ("腎臓", "腎臓病", "腎臓科", "腎臓病"): ['腎臓・内分泌内科', '腎臓・内分泌外科'],
    "甲状腺診療科": ['腎臓・内分泌内科', '腎臓・内分泌外科'],
    "血液浄化": ['腎臓・内分泌内科'],
    ("乳腺内", "乳内", "内乳"): ["腎臓・内分泌内科"],
    ("乳腺", "「乳腺", "乳", "乳腺科", "胸部・乳", "乳内外"): ['腎臓・内分泌内科', '腎臓・内分泌外科'],
    ("透析", "透析科", "人工透析", "人工透析科", "人口透析科", "通院人工透析診療所", "循環器科（透析）") : ['腎臓・内分泌内科'],
    ("高内"): ["腎臓・内分泌内科"], # おそらく「高血圧・内分泌内科」 Hypertension and Endocrinology
    "小腎": ['小児科', "腎臓・内分泌内科"], # おそらく 小児腎臓内科 or 小児腎臓外科 Pediatric Nephrology or Pediatric Kidney Surgery
    "内高精": ["腎臓・内分泌内科", "泌尿器内科"], # おそらく 内分泌・高血圧・生殖内科 or 生殖医療科 (精 = 精巣機能：男性不妊？)   Endocrinology, Hypertension, and Reproductive Medicine / Reproductive Medicine (related to male infertility?)
    ("泌尿器科（人工透析）", "泌尿器科・泌尿器科（人工透析）", "腎臓・泌尿器科", "泌尿器科(人工透析)", "泌尿器科(人工透析）", "泌尿器科（透析）", "腎臓泌尿器科", "人工透析・泌尿器科", "泌尿器科（腎臓）", "ひ尿器科（人工透析）", "透析泌尿器科", "腎・泌尿器科（人工透析）", "腎泌尿器科", "泌尿器科（ 人工透析 ）", "泌尿器科（人口透析）"): ["腎臓・内分泌内科", "泌尿器内科", "泌尿器外科"],

    # ----泌尿器科----
    ("泌尿器", "泌尿器科", "女性泌尿器科", "肛門科", "ひ尿器科", "老年泌尿器科", "泌尿器科科", "男性泌尿器科", "ひ尿科", "泌尿科", "泌尿器科（生）", "漢方泌尿器科", "老年泌尿器科", "泌尿器科（生殖医療）"): ['泌尿器内科', '泌尿器外科'],

    # ----大腸・肛門----
    ("こうもん科", "肛門科", "大腸・肛門科" ,"大腸肛門科", "こう門科", "胃腸科肛門科", "大腸", "肛門科（マンモグラフィあり）"): ["大腸・肛門内科", "大腸・肛門外科"],
    ("消化器科・肛門科"): ["消化器内科", "消化器外科", "大腸・肛門内科", "大腸・肛門外科"],
    ("肛門下科"): ["大腸・肛門外科"],
    ("乳腺・肛門科"): ["腎臓・内分泌内科", "腎臓・内分泌外科", "大腸・肛門内科", "大腸・肛門外科"],

    # ----耳鼻咽喉----
    ("耳鼻いんこう科", "耳鼻咽喉科" ,"耳鼻科", "耳鼻咽喉科（補聴器相談室）", "耳鼻いんこう科（耳鼻科）", "漢方耳鼻咽喉科", "耳鼻咽頭科", "頭頸部・耳鼻いんこう科", "耳咽咽喉科"): ["耳鼻咽喉・頭頸部内科", "耳鼻咽喉・頭頸部外科"],
    ("気管食道耳鼻いんこう科", "気管食道・耳鼻咽喉科", "気管食道耳鼻咽喉科", "気管食道・耳鼻いんこう科", "耳咽・気管食道科"): ["気管食道内科", "気管食道外科", "耳鼻咽喉・頭頸部内科", "耳鼻咽喉・頭頸部外科"],

    # ----others----
    ('神経泌尿器科'): ["泌尿器内科", "泌尿器外科", "脳神経内科", "脳神経外科"],
    ("神経科", "脳神経科", "神経痛"): ["脳神経内科", "脳神経外科"],
    ("脳・血管内科"): ["脳血管内科"],
    ("がん"): ["血液・腫瘍内科", "血液・腫瘍外科"],
    ('腫瘍泌尿器科'): ["泌尿器内科", "泌尿器外科", "血液・腫瘍内科", "血液・腫瘍外科"],
}

In [3]:
# load data
uniq_dpt = pd.read_csv('2014_v2.csv').iloc[:,1]
# uniq_dpt = pd.read_csv('2014.csv').iloc[:,1]
uniq_dpt = uniq_dpt.to_list()
uniq_dpt

['内科',
 '神経内科',
 '小児科',
 '外科',
 '整形外科',
 '脳神経外科',
 '呼吸器外科',
 '心臓血管外科',
 '小児外科',
 '泌尿器科',
 '産婦人科',
 '眼科',
 'リハビリテーション科',
 '放射線科',
 '麻酔科',
 '呼吸器内科',
 '循環器内科',
 '消化器内科',
 '消化器外科',
 '乳腺外科',
 '甲状腺外科',
 '病理診断科',
 '緩和ケア科',
 '精神科',
 '形成外科',
 '皮膚科',
 '耳鼻いんこう科',
 '歯科',
 '矯正歯科',
 '歯科口腔外科',
 'リウマチ科',
 '消化器科',
 '心療内科',
 '腫瘍外科',
 '血液内科',
 '腫瘍内科',
 '呼吸器科',
 '循環器科',
 'こう門科',
 '肛門外科',
 '腎臓内科',
 '糖尿病内科',
 '代謝内科',
 '緩和ケア内科',
 '内視鏡内科',
 '老年内科',
 '呼吸器リハビリテーション科',
 '児童精神科',
 '老年精神科',
 '大腸外科',
 '小児皮膚科',
 '内分泌内科',
 '神経科',
 '胃腸科',
 'アレルギー科',
 '産科',
 '婦人科',
 '脳神経内科',
 'こう門外科',
 '小児歯科',
 '人工透析内科',
 '小児神経科',
 '心臓血管内科',
 '糖尿病内分泌内科',
 '内視鏡外科',
 '頭頚部外科',
 '血管外科',
 '糖尿病・内分泌内科',
 '化学療法内科',
 '糖尿病・代謝内科',
 '血液・腫瘍内科',
 '胃腸内科（内視鏡）',
 'ペインクリニック内科',
 '内視鏡外科（大腸・胃）',
 '内分泌外科',
 '救急科',
 '放射線診断科',
 '胃腸内科',
 '循環器外科',
 '乳腺・内分泌内科',
 '呼吸気内科',
 '神経精神科',
 '人工透析外科',
 '気管食道科',
 '小児科（感染症・アレルギー疾患）',
 'ペインクリニック外科',
 '心臓内科',
 '腎臓内科（人工透析）',
 '老年脳神経外科',
 '放射線治療科',
 '胃・大腸・肛門外科',
 '糖尿病代謝内科',
 '腎臓移植外科',
 '頭頸部外科',
 '気管食道外科',
 '腫瘍整形外科',
 '胸部外科',
 

In [4]:
classified_keywords = classify_keywords(
    keywords=uniq_dpt,               # List of unique medical department keywords to be classified
    classification_dict=dict.classification_dict,  # Dictionary defining classification rules for keywords
    multiple=True,                   # If True, allows matching with multiple classifications
    override_map=override_map_2014,   # Dictionary containing individual classification rules
    override_match_type="exact"       # Matching type for override_map classification ("exact" means exact match)
)

# Display the result
max_length = max(len(values) for values in classified_keywords.values())
data = {
    cat: values + [""] * (max_length - len(values))
    for cat, values in classified_keywords.items()
}
result_df = pd.DataFrame(data)
display(result_df)

,内科系,一般内科,呼吸器内科,気管食道内科,耳鼻咽喉・頭頸部内科,消化器内科,循環器内科,腎臓・内分泌内科,糖尿病・代謝内科,血液・腫瘍内科,...,小児外科,産婦人科,精神科,放射線科,救急・集中治療科,臨床検査科,病理診断科,緩和ケア科,鍼灸院,分類不可
0,内科,内科,呼吸器内科,呼吸気内科,耳鼻いんこう科,消化器内科,循環器内科,腎臓内科,糖尿病内科,血液内科,...,小児外科,産婦人科,精神科,放射線科,救急科,化学療法内科,病理診断科,緩和ケア科,指圧,不明
1,神経内科,総合内科,呼吸器科,気管食道科,耳鼻咽喉科,消化器科,循環器科,内分泌内科,代謝内科,腫瘍内科,...,小児心臓血管外科,産科,児童精神科,放射線診断科,小児救急科,臨床検査科,理学診療科,緩和ケア内科,はり,診療内
2,呼吸器内科,人間ドック,呼吸気内科,気管食道耳鼻いんこう科,気管食道耳鼻いんこう科,胃腸科,小児循環器内科,人工透析内科,糖尿病内分泌内科,血液・腫瘍内科,...,小児脳神経外科,婦人科,老年精神科,放射線治療科,,がん化学療法外科,病理診療科,疼痛緩和内科,,医科
3,循環器内科,一般内科,内科（消化器・循環器・呼吸器・内分泌・代謝・糖尿病・胃腸・大腸内視鏡・老人・人工透析・漢方）,気管食道内科,耳鼻科,胃腸内科（内視鏡）,内科（消化器・循環器・呼吸器・内分泌・代謝・糖尿病・胃腸・大腸内視鏡・老人・人工透析・漢方）,糖尿病内分泌内科,糖尿病・内分泌内科,血液腫瘍内科,...,小児形成外科,婦人科（生殖医療）,神経精神科,腫瘍放射線科,,病理診断・臨床検査科,理学療法科,緩和医療科,,NaN
4,消化器内科,総合診療科,内科（呼吸器・消化器・心療）,気道食道科,耳鼻咽喉科（補聴器相談室）,胃腸内科,内科（循環器）,糖尿病・内分泌内科,糖尿病・代謝内科,小児血液腫瘍内科,...,小児科外科,産婦人科（産科）,小児精神科,神経放射線科,,外科（化学療法）,病理診断・臨床検査科,緩和ケア,,他
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
464,内科（代謝）,,,,,,,,,,...,,,,,,,,,,
465,内科（性感染症）,,,,,,,,,,...,,,,,,,,,,
466,内科（老年）,,,,,,,,,,...,,,,,,,,,,
467,内科（脂質代謝）,,,,,,,,,,...,,,,,,,,,,


In [5]:
result_df.to_csv('../classification_results/2014_classified.csv', encoding='utf-8', index=True)